# Import Libraries

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD

import cv2

# Data Preparation

In [0]:
mnist = tf.keras.datasets.mnist  # load mnist dataset from tensorflow
(input_train, output_train_class), (input_test, output_test_class) = mnist.load_data()

input_train= input_train.reshape(input_train.shape[0], 28*28, 1) 
input_test= input_test.reshape(input_test.shape[0], 28*28, 1)

input_train = input_train / 255.0  # max normalise the image data[0:1]
input_test = input_test / 255.0

# Exercise 1

In [0]:
optimizer_type = SGD(lr=0.2)  # optimisation algorithm: SGD stochastic gradient decent 
loss = 'mean_squared_error'
metrics = ['mean_absolute_error']  # network accuracy metric to be determined after each epoch
dropout_ratio = 0.0  # % of nodes in the hidden layer to dropout during back-propagation update of the network weights
validtrain_split_ratio = 0.2  # % of the seen dataset to be put aside for validation, rest is for training
max_epochs = 1000  # maxmimum number of epochs to be iterated
batch_size = 300   # batch size for the training data set
batch_shuffle = True   # shuffle the training data prior to batching before each epoch
num_hidden_nodes = 200  # number of nodes in hidden fully connected layer

input_shape = (28*28, 1)
inputs = Input(shape=input_shape)

flatten = Flatten()(inputs)   # 784 nodes = 28x28

dense_01 = Dense(num_hidden_nodes)(flatten)
dense_01 = Activation('sigmoid')(dense_01)
dense_01 = Dropout(dropout_ratio)(dense_01)

dense_02 = Dense(28*28)(dense_01)
outputs = Activation('sigmoid')(dense_02)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizer_type, loss=loss, metrics=metrics)

history = model.fit(input_train, input_train.reshape(len(input_train),28*28),\
                    batch_size=batch_size, \
                    epochs=max_epochs, \
                    validation_split=validtrain_split_ratio, \
                    shuffle=batch_shuffle)

In [0]:
sort=np.argsort(output_test_class)
test=input_test[sort]
output_test=output_test_class[sort]

_,i=np.unique(output_test,True)
imgs=test[i].reshape(test[i].shape[0],28,28)
preds=model.predict(test[i].reshape(test[i].shape[0],28*28,1))
preds=preds.reshape(preds.shape[0],28,28)
for i in range(10):
  plt.figure(i)
  plt.subplot(121);plt.imshow(imgs[i],cmap='viridis');plt.grid(False)
  plt.subplot(122);plt.imshow(preds[i],cmap='viridis');plt.grid(False)

# Exercises 2 and 3

In [0]:
num_hidden_nodes = 500  # number of nodes in hidden fully connected layer

input_shape = (28*28, 1)
inputs = Input(shape=input_shape)

flatten = Flatten()(inputs)   # 784 nodes = 28x28

dense_01 = Dense(num_hidden_nodes)(flatten)
dense_01 = Activation('sigmoid')(dense_01)
dense_01 = Dropout(dropout_ratio)(dense_01)

dense_02 = Dense(28*28)(dense_01)
outputs = Activation('sigmoid')(dense_02)

model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizer_type, loss=loss, metrics=metrics)

blurredImgs=[]
for i in range(input_train.shape[0]):
  blurredImgs.append(cv2.GaussianBlur(input_train[i],(19,19),0))
blurredImgs=np.array(blurredImgs)
blurredImgs=blurredImgs.reshape(blurredImgs.shape[0],28*28,1)

history = model.fit(blurredImgs, input_train.reshape(len(input_train),28*28),\
                    batch_size=batch_size, \
                    epochs=max_epochs, \
                    validation_split=validtrain_split_ratio, \
                    shuffle=batch_shuffle)

In [0]:
sort=np.argsort(output_test_class)
test=input_test[sort]
output_test=output_test_class[sort]

blurredTestImgs=[]
for i in range(test.shape[0]):
  blurredTestImgs.append(cv2.GaussianBlur(test[i],(19,19),0))
blurredTestImgs=np.array(blurredTestImgs)
blurredTestImgs=blurredTestImgs.reshape(blurredTestImgs.shape[0],28*28,1)

_,i=np.unique(output_test,True)
imgs=test[i].reshape(test[i].shape[0],28,28)
imgsBlurred=blurredTestImgs[i].reshape(blurredTestImgs[i].shape[0],28,28)
preds=model.predict(blurredTestImgs[i].reshape(blurredTestImgs[i].shape[0],28*28,1))
preds=preds.reshape(preds.shape[0],28,28)
for i in range(10):
  plt.figure(i)
  plt.subplot(131);plt.imshow(imgs[i],cmap='viridis');plt.grid(False)
  plt.subplot(132);plt.imshow(imgsBlurred[i],cmap='viridis');plt.grid(False)
  plt.subplot(133);plt.imshow(preds[i],cmap='viridis');plt.grid(False)